### Data Merging Basics

In [1]:
import pandas as pd
import numpy as np

## Inner join
You have been tasked with figuring out what the most popular types of fuel used in Chicago taxis are. To complete the analysis, you need to merge the `taxi_owners` and `taxi_veh` tables together on the `vid` column. You can then use the merged table along with the `.value_counts()` method to find the most common `fuel_type`.

In [3]:
taxi_owners = pd.read_pickle("datasets/taxi_owners.p")
taxi_owners.head()

,rid,vid,owner,address,zip
0,T6285,6285,AGEAN TAXI LLC,4536 N. ELSTON AVE.,60630
1,T4862,4862,MANGIB CORP.,5717 N. WASHTENAW AVE.,60659
2,T1495,1495,"FUNRIDE, INC.",3351 W. ADDISON ST.,60618
3,T4231,4231,ALQUSH CORP.,6611 N. CAMPBELL AVE.,60645
4,T5971,5971,EUNIFFORD INC.,3351 W. ADDISON ST.,60618


In [4]:
taxi_veh = pd.read_pickle("datasets/taxi_vehicles.p")
taxi_veh.head()

,vid,make,model,year,fuel_type,owner
0,2767,TOYOTA,CAMRY,2013,HYBRID,SEYED M. BADRI
1,1411,TOYOTA,RAV4,2017,HYBRID,DESZY CORP.
2,6500,NISSAN,SENTRA,2019,GASOLINE,AGAPH CAB CORP
3,2746,TOYOTA,CAMRY,2013,HYBRID,"MIDWEST CAB CO, INC"
4,5922,TOYOTA,CAMRY,2013,HYBRID,SUMETTI CAB CO


In [8]:
taxi_owners.merge(taxi_veh, on="vid", suffixes=["_own","_veh"])["fuel_type"].value_counts()

HYBRID                    2792
GASOLINE                   611
FLEX FUEL                   89
COMPRESSED NATURAL GAS      27
Name: fuel_type, dtype: int64

### Inner joins and number of rows returned

In [9]:
wards = pd.read_pickle("datasets/ward.p")
census = pd.read_pickle("datasets/census.p")

In [10]:
wards.shape

(50, 4)

In [11]:
census.shape

(50, 6)

In [12]:
wards.head()

,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609
3,4,William D. Burns,"435 EAST 35TH STREET, 1ST FLOOR",60616
4,5,Leslie A. Hairston,2325 EAST 71ST STREET,60649


In [13]:
census.head()

,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,40385,53039,31%,17 EAST 38TH STREET,60653
3,4,51953,54589,5%,31ST ST HARBOR BUILDING LAKEFRONT TRAIL,60653
4,5,55302,51455,-7%,JACKSON PARK LAGOON SOUTH CORNELL DRIVE,60637


* Merge wards and census on the ward column and save the result to wards_census.

In [15]:
wards_census = wards.merge(census, on="ward")
wards_census.head(3)

,ward,alderman,address_x,zip_x,pop_2000,pop_2010,change,address_y,zip_y
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622
2,3,Pat Dowell,5046 SOUTH STATE STREET,60609,40385,53039,31%,17 EAST 38TH STREET,60653


### One-to-many merge

- Starting with the licenses table on the left, merge it to the biz_owners table on the column account, and save the results to a variable named licenses_owners.

In [20]:
licenses = pd.read_pickle("datasets/licenses.p")
biz_owners = pd.read_pickle("datasets/business_owners.p")

display(licenses.shape, biz_owners.shape)
display(licenses.head(2), biz_owners.head(2))

(10000, 6)

(21352, 4)

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633


,account,first_name,last_name,title
0,10,PEARL,SHERMAN,PRESIDENT
1,10,PEARL,SHERMAN,SECRETARY


In [23]:
licenses_owners = licenses.merge(biz_owners, on = "account")
licenses_owners.shape

(19497, 9)

- Group licenses_owners by title and count the number of accounts for each title. Save the result as counted_df

In [41]:
counted_df = licenses_owners.groupby("title").agg({'account':'count'})
counted_df.head(3)

,account
title,
ASST. SECRETARY,111
BENEFICIARY,4
CEO,110


- Sort counted_df by the number of accounts in descending order, and save this as a variable named sorted_df.

In [42]:
sorted_df = counted_df.sort_values(by="account", ascending=False)

- Use the .head() method to print the first few rows of the sorted_df.

In [43]:
sorted_df.head(3)

,account
title,
PRESIDENT,6259
SECRETARY,5205
SOLE PROPRIETOR,1658


### Merging multiple dataframes

- Merge the ridership and cal tables together, starting with the ridership table on the left and save the result to the variable ridership_cal. If you code takes too long to run, your merge conditions might be incorrect.

In [49]:
cal = pd.read_pickle("datasets/cta_calendar.p")
ridership = pd.read_pickle("datasets/cta_ridership.p")
stations = pd.read_pickle("datasets/stations.p")

display(cal.shape, ridership.shape, stations.shape)
display(cal.head(2), ridership.head(2), stations.head(2))

(365, 4)

(3285, 5)

(144, 3)

,year,month,day,day_type
0,2019,1,1,Sunday/Holiday
1,2019,1,2,Weekday


,station_id,year,month,day,rides
0,40010,2019,1,1,576
1,40010,2019,1,2,1457


,station_id,station_name,location
0,40010,Austin-Forest Park,"(41.870851, -87.776812)"
1,40020,Harlem-Lake,"(41.886848, -87.803176)"


In [52]:
ridership_cal = ridership.merge(cal, on=["year", "month", "day"])
ridership_cal.shape, ridership.head(3)

((3285, 6),
   station_id  year  month  day  rides
 0      40010  2019      1    1    576
 1      40010  2019      1    2   1457
 2      40010  2019      1    3   1543)

- Extend the previous merge to three tables by also merging the stations table.

In [57]:
ridership_cal_stations = ridership.merge(cal, on=['year','month','day']) \
                         .merge(stations, on = "station_id")
ridership_cal_stations.head(3)

,station_id,year,month,day,rides,day_type,station_name,location
0,40010,2019,1,1,576,Sunday/Holiday,Austin-Forest Park,"(41.870851, -87.776812)"
1,40010,2019,1,2,1457,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"
2,40010,2019,1,3,1543,Weekday,Austin-Forest Park,"(41.870851, -87.776812)"


- Your goal is to find the total number of rides provided to passengers passing through the Wilson station (station_name == 'Wilson') when riding Chicago's public transportation system on weekdays (day_type == 'Weekday') in July (month == 7)
- Create a variable called filter_criteria to select the appropriate rows from the merged table so that you can sum the rides column.

In [60]:
filter_criteria = (ridership_cal_stations["station_name"] == "Wilson") & \
                 (ridership_cal_stations["day_type"] == "Weekday") & \
                 (ridership_cal_stations["month"] == 7)

In [63]:
ridership_cal_stations.loc[filter_criteria, "rides"].sum()

140005

### Three table merge

- Starting with the licenses table, merge to it the zip_demo table on the zip column. Then merge the resulting table to the wards table on the ward column. Save result of the three merged tables to a variable named licenses_zip_ward.

In [65]:
licenses = pd.read_pickle("datasets/licenses.p")
zip_demo = pd.read_pickle("datasets/zip_demo.p")
wards = pd.read_pickle("datasets/ward.p")

display(licenses.head(2), zip_demo.head(2), wards.head(2))

,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633


,zip,income
0,60630,70122
1,60640,50488


,ward,alderman,address,zip
0,1,"Proco ""Joe"" Moreno",2058 NORTH WESTERN AVENUE,60647
1,2,Brian Hopkins,1400 NORTH ASHLAND AVENUE,60622


In [68]:
licenses_zip_ward = licenses.merge(zip_demo, on="zip").merge(wards, on="ward")
licenses_zip_ward.head(3)

,account,ward,aid,business,address_x,zip_x,income,alderman,address_y,zip_y
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616,46340,Pat Dowell,5046 SOUTH STATE STREET,60609
1,11280,3,763,PRIME WAY,2251 S STATE ST 1ST,60616,46340,Pat Dowell,5046 SOUTH STATE STREET,60609
2,15015,3,NaN,"SOUTHVIEW MANOR, INC.",3311 S MICHIGAN AVE,60616,46340,Pat Dowell,5046 SOUTH STATE STREET,60609


- Group the results of the three merged tables by the column alderman and find the median income.

In [70]:
licenses_zip_ward.groupby("alderman").agg({"income": "median"}).head()

,income
alderman,
Ameya Pawar,66246.0
Anthony A. Beale,38206.0
Anthony V. Napolitano,82226.0
Ariel E. Reyboras,41307.0
Brendan Reilly,110215.0


### One-to-many merge

- Merge land_use and census on the ward column. Merge the result of this with licenses on the ward column, using the suffix _cen for the left table and _lic for the right table. Save this to the variable land_cen_lic.

In [71]:
land_use =pd.read_pickle("datasets/land_use.p")
census = pd.read_pickle("datasets/census.p")
licenses = pd.read_pickle("datasets/licenses.p")

display(land_use.head(2), census.head(2), licenses.head(2))

,ward,residential,commercial,industrial,vacant,other
0,1,41,9,2,2,46
1,2,31,11,6,2,50


,ward,pop_2000,pop_2010,change,address,zip
0,1,52951,56149,6%,2765 WEST SAINT MARY STREET,60647
1,2,54361,55805,3%,WM WASTE MANAGEMENT 1500,60622


,account,ward,aid,business,address,zip
0,307071,3,743,REGGIE'S BAR & GRILL,2105 S STATE ST,60616
1,10,10,829,HONEYBEERS,13200 S HOUSTON AVE,60633


In [72]:
land_use.shape, census.shape, licenses.shape

((50, 6), (50, 6), (10000, 6))

In [78]:
land_cen_lic = land_use.merge(census, on="ward").merge(licenses, on="ward", suffixes=["_cen", "_lic"])

- Group land_cen_lic by ward, pop_2010 (the population in 2010), and vacant, then count the number of accounts. Save the results to pop_vac_lic.

In [79]:
land_cen_lic.head(1)

,ward,residential,commercial,industrial,vacant,other,pop_2000,pop_2010,change,address_cen,zip_cen,account,aid,business,address_lic,zip_lic
0,1,41,9,2,2,46,52951,56149,6%,2765 WEST SAINT MARY STREET,60647,12024,NaN,DIGILOG ELECTRONICS,1038 N ASHLAND AVE,60622


In [85]:
pop_vac_lic = land_cen_lic.groupby(["ward", "pop_2010", "vacant"], as_index=False).agg({"account":"count"})
pop_vac_lic.head()

,ward,pop_2010,vacant,account
0,1,56149,2,253
1,10,51535,14,130
2,11,51497,5,201
3,12,52235,4,255
4,13,53722,1,101


- Sort pop_vac_lic by vacant, account, and pop_2010 in descending, ascending, and ascending order respectively. Save it as sorted_pop_vac_lic.

In [90]:
sorted_pop_vac_lic = pop_vac_lic.sort_values(by=["vacant", "account", "pop_2010"], ascending=[False, True, True])
sorted_pop_vac_lic.head(2)

,ward,pop_2010,vacant,account
47,7,51581,19,80
12,20,52372,15,123
